In [1]:
import torch
import torch.nn.functional as F

from jamba import JambaLMConfig, JambaLM

In [17]:
config = JambaLMConfig(d_model=64, n_layers=8, mlp_size=128, vocab_size=60, attn_layer_period=1, attn_layer_offset=0)
# 100, 4 = full mamba
# 1, 0 = full attention
model = JambaLM(config)
model = model.eval()

In [22]:
def gen_no_caching(num_tokens):
    batch_size = 1
    
    input_ids = 2*torch.ones(batch_size, 1, dtype=torch.int64)

    for i in range(num_tokens):
        next_token_logits, _ = model(input_ids[:, 0:i+1])
        probs = F.softmax(next_token_logits[:, [-1]], dim=-1)
        next_token = torch.argmax(probs, dim=-1)

        input_ids = torch.cat([input_ids, next_token], dim=1)

    return input_ids

In [23]:
gen_no_caching(4)

tensor(0.0408, grad_fn=<MeanBackward0>)
tensor(0.0367, grad_fn=<MeanBackward0>)
tensor(0.0347, grad_fn=<MeanBackward0>)
tensor(0.0334, grad_fn=<MeanBackward0>)


tensor([[ 2, 21, 21, 21, 21]])

In [24]:
def gen(num_tokens):
    batch_size = 1

    caches = [model.jamba.layers[i].get_empty_cache(batch_size) for i in range(config.n_layers)]
    
    input_ids = 2*torch.ones(batch_size, 1, dtype=torch.int64)

    for i in range(num_tokens):
        next_token_logits, caches = model(input_ids[:, [i]], caches)
        probs = F.softmax(next_token_logits, dim=-1)
        next_token = torch.argmax(probs, dim=-1)

        input_ids = torch.cat([input_ids, next_token], dim=1)

    return input_ids

In [25]:
gen(4)

tensor(0.0408, grad_fn=<MeanBackward0>)
tensor(0.0359, grad_fn=<MeanBackward0>)
tensor(0.0359, grad_fn=<MeanBackward0>)
tensor(0.0359, grad_fn=<MeanBackward0>)


tensor([[ 2, 21, 21, 21, 21]])